# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [23]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/
https://edwarddonner

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
anthropic = Website("https://anthropic.com")
anthropic.links

['/',
 '/claude',
 '/team',
 '/enterprise',
 '/api',
 '/pricing',
 '/research',
 '/company',
 '/careers',
 '/news',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/3-5-models-and-computer-use',
 '/claude/sonnet',
 '/claude/haiku',
 '/news/claude-for-enterprise',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/news/core-views-on-ai-safety',
 '/jobs',
 '/',
 '/claude',
 '/api',
 '/team',
 '/pricing',
 '/research',
 '/company',
 '/customers',
 '/news',
 '/careers',
 'mailto:press@anthropic.com',
 'https://support.anthropic.com/',
 'https://status.anthropic.com/',
 '/supported-countries',
 'https://twitter.com/AnthropicAI',
 'https://www.linkedin.com/company/anthropicresearch',
 'https://www.youtube.com/@anthropic-ai',
 '/legal/consumer-terms',
 '/legal/commercial-terms',
 '/legal/privacy',
 '/legal/aup',
 '/responsible-disclosure-policy',
 'https://trust.anthropic.com/']

In [11]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://anthropic.com/team'},
  {'type': 'api page', 'url': 'https://anthropic.com/api'},
  {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'},
  {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'},
  {'type': 'research page', 'url': 'https://anthropic.com/research'},
  {'type': 'news page', 'url': 'https://anthropic.com/news'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
Learn more
Announcements
Introducing computer use, a new 

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'products page', 'url': 'https://www.anthropic.com/claude'}, {'type': 'enterprise page', 'url': 'https://www.anthropic.com/enterprise'}, {'type': 'API page', 'url': 'https://www.anthropic.com/api'}]}


"You are looking at a company called: Anthropic\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nClaude\nOverview\nTeam\nEnterprise\nAPI\nPricing\nResearch\nCompany\nCareers\nNews\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.5 Sonnet\nClaude 3.5 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nStart using Claude to drive efficiency and create new revenue streams.\nLearn more\nAnnouncements\nIntroducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku\nOct 22, 2024\nModel updates\n3.5 Sonnet\n3.5 Haiku\nOur Work\nProduct\nClaude for Enterprise\nSep 4, 2024\nAlignment\n·\nResearch\nConstitutional AI: Harmlessness from AI Feedback\nDec 15, 2022\nAnnouncements\nCore Views on AI Safety: When, Why, What, and How\nMar 8, 2023\nWo

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}]}


# Anthropic Brochure

---

## About Anthropic

### Our Mission
At **Anthropic**, we are an AI safety and research company dedicated to building reliable, interpretable, and steerable AI systems. Our commitment is fueled by the belief that AI can profoundly impact the world. We focus on ensuring that these systems are safe and contribute positively to society.

### Our Work
With our flagship AI model, **Claude**, we strive to develop cutting-edge technology that empowers organizations to realize the benefits of AI while prioritizing safety. From our research initiatives to our enterprise solutions, we aim to create beneficial AI systems that people can rely on.

---

## Company Culture

### Values
Our culture is built on four core values:

1. **Here for the Mission**: We pursue our mission with a focus on building systems that help people flourish.
2. **Unusually High Trust**: We foster an environment of trust, kindness, and honesty where emotional maturity is valued.
3. **One Big Team**: Collaboration is at the heart of our operations. We work together towards shared goals, leveraging diverse expertise.
4. **Do the Simple Thing That Works**: We embrace practicality and empirical approaches, preferring straightforward solutions over overly complex ones.

### Interdisciplinary Team
Our team comprises researchers, engineers, policy experts, and operational leaders from diverse backgrounds. This interdisciplinary approach helps us assess AI's impacts and safety, promoting collective engagement in steering safe AI development.

---

## Customers

Anthropic works with a variety of clients ranging from businesses to nonprofits, aiming to integrate AI safely across sectors. Our solutions are designed to enhance productivity, drive efficiency, and provide valuable insights that empower organizations.

### Notable Testimonials
- "Claude is a truly exceptional writer that has helped our team complete tasks up to 5x faster." — Luka Anic, Senior Director at North Highland
- "Generating content and creating summarized reports is much easier than before." — Olga Pirog, Head of AI Transformation at IG Group

---

## Careers at Anthropic

### Join Our Team
We're looking for passionate individuals who are excited about the future of AI and are dedicated to ensuring its safe development. With a team culture that values collaboration and innovation, Anthropic is a great place to build a career.

### What We Offer
- Comprehensive health, dental, and vision benefits 
- Flexible paid time off and 22 weeks of paid parental leave 
- Generous mental health support and wellness stipends
- Competitive salaries and equity packages

### Hiring Process
Our interview process focuses on evaluating skills and fit through a sequence of exploratory talks, skills assessments, and collaborative discussions. We value all backgrounds and encourage applicants with diverse experiences, even those without formal machine learning training.

### Current Openings
Explore our open roles [here](#) and help us shape the future of AI!

---

## Get In Touch
For inquiries about partnerships, products, or careers with Anthropic, feel free to reach out through our [official website](#).

---

### Connect with Us
- [Twitter](#)
- [LinkedIn](#)
- [YouTube](#)

**Together, we can build AI systems that put safety at the forefront of innovation!**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}


# Anthropic Brochure

## Welcome to Anthropic

Anthropic is an AI safety and research company based in San Francisco, dedicated to building reliable, interpretable, and steerable AI systems. We believe AI will have a transformative impact on the world, and our mission is to develop systems that people can trust, while conducting research that addresses the opportunities and risks associated with advanced AI.

### Our AI Product: Claude

Meet **Claude** - our flagship AI model. With the latest versions, including Claude 3.5 Sonnet and Haiku, we offer dynamic solutions for businesses to enhance efficiency, drive revenue, and transform workflows. Claude acts as a virtual teammate, optimizing and assisting in various tasks, from writing and coding to generating insights from data.

### Our Commitment to Safety

At Anthropic, we treat AI safety as a systematic science. We conduct rigorous research on alignment and safety techniques, ensuring that the systems we build are not only innovative but also align with our ethical standards for societal benefit. Our interdisciplinary team collaborates closely across fields such as machine learning (ML), physics, policy, and product development.

### A Culture of Trust and Collaboration

Our company culture is defined by:

- **High Trust:** We maintain an environment that values honesty and emotional maturity.
- **Teamwork:** Everyone at Anthropic works together towards our mission, contributing across various teams and areas of expertise.
- **Simplicity:** We prioritize straightforward, pragmatic solutions that deliver results efficiently while embracing empirical evidence.

### Customers

Anthropic collaborates with various sectors, including businesses, nonprofits, and civil society organizations, to translate our research into practical tools. Our technology is designed to benefit users globally, enabling better decision-making and productivity.

### Join Us: Careers at Anthropic

We are constantly looking for passionate individuals to join our mission. We offer roles for researchers, engineers, and operations staff, welcoming diverse backgrounds and expertise. Our hiring process emphasizes equity and offers a structured assessment to ensure we recognize talent beyond traditional metrics.

#### Employee Benefits

At Anthropic, we prioritize our team’s well-being with a comprehensive benefits package that includes:

- **Health & Wellness:** Comprehensive health, dental, and vision insurance, inclusive fertility benefits, and robust mental health support.
- **Work-Life Balance:** Flexible paid time off, 22 weeks of paid parental leave, and support for relocation.
- **Growth Opportunities:** Annual education stipends, professional development supports, and a culture of collaboration.

**Ready to make an impact?** Explore open roles on our [Careers page](#).

### Connect With Us

Stay updated with our latest research, product announcements, and team highlights through our various channels:

- [LinkedIn](#)
- [Twitter](#)
- [YouTube](#)

---

At Anthropic, we are paving the way for safe and beneficial AI that can help humanity thrive in the face of rapid technological advancement. Together, let's build a future we can all rely on.

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

## About Us
Hugging Face is a pioneering company in the Artificial Intelligence (AI) and Machine Learning (ML) community. With a mission to democratize machine learning, we provide a platform for collaboration, experimentation, and the development of cutting-edge technologies. Our community of over **50,000 organizations**, including giants like Microsoft, Google, and Amazon, collaborates on models, datasets, and applications that shape the future of AI.

---

## Our Offerings
### Collaboration Platform
- Host unlimited models, datasets, and applications.
- Engage with **400,000+ models** and **100,000+ datasets**.
- Share your work and build your portfolio in a vibrant community.

### Enterprise Solutions
- **Enterprise Hub** starting at **$20/user/month** with advanced security and support.
- Options for custom compute resources and dedicated endpoints.
- More than 50,000 organizations trust Hugging Face for their AI needs.

### Open Source Tools
Our open-source framework includes:
- **Transformers** for state-of-the-art ML.
- **Diffusers** for image and audio generation.
- Libraries for tokenization, parameter-efficient fine-tuning, and more.

---

## Company Culture
At Hugging Face, our team of **over 224 members** is dedicated to collaboration, innovation, and inclusivity. We believe in creating a welcoming environment where everyone's ideas contribute to our mission. Join us in building a future where machine learning is accessible to all.

### Careers
We are always on the lookout for passionate individuals to join our team. Check our **[Current Openings](#)** page to find roles in engineering, research, community management, and more. If you have a commitment to advancing AI, we want to hear from you!

---

## Our Community
We are proud to support a vibrant community that helps to pave the way for advancements in multiple domains including:
- Natural Language Processing (NLP)
- Computer Vision (CV)
- Reinforcement Learning (RL)

We encourage collaboration and sharing of knowledge through our **Hugging Face Blog**, articles, and community discussions.

---

## Why Choose Hugging Face?
- **Innovative Community**: Collaborate with leading experts and organizations.
- **Robust Resources**: Access advanced tools and enterprise solutions tailored for your needs.
- **Commitment to Excellence**: At Hugging Face, we prioritize quality, security, and customer satisfaction.

---

**Join us in building the future of AI! Visit our website [huggingface.co](https://huggingface.co) to learn more, explore our offerings, and become part of our community.**

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>